In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime
import statistics
from math import sqrt
from numpy import log as ln
from tqdm import tqdm
#from pymongo import MongoClient

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/productscout_db1.csv", index_col = [0])

In [ ]:
df["Number of Reviews"] = df["Number of Reviews"].str.replace("Ratings","")
df["Number of Reviews"] = df["Number of Reviews"].str.replace("No","0")
df['Number of Reviews'] = df['Number of Reviews'].astype('int')
df["Score"] = df["Score"].str.replace("/5","")
df['Score'] = df['Score'].astype('float')

In [ ]:
data = pd.DataFrame()
data["Categoric Mapping"] = "Categoric Mapping"
data["Categoric Mapping"] = df["Date"] + "_" + df["Category"] + "/" + df[ "Sub-Category"] + "/" + df["SubSub-Category"]
data["Mean Reviews"] = df["Number of Reviews"]
data["Mean Score"] = df["Score"]
data["Popularity"] = df["Popularity"]

In [ ]:
value_counts = dict(data["Categoric Mapping"].value_counts())

In [ ]:
data = data.groupby("Categoric Mapping").agg({"Mean Reviews":"sum", 
                                              "Mean Score":"sum", 
                                              "Popularity":"max"})
data.reset_index(inplace = True)

In [ ]:
keys = list(value_counts.keys())
for i in tqdm(range(len(data))):
  temp = data["Categoric Mapping"][i]
  try:
    data["Mean Reviews"][i] = data["Mean Reviews"][i] / value_counts[temp]
    data["Mean Score"][i] = data["Mean Score"][i] / value_counts[temp]
    # data["Popularity"][i] = data["Popularity"][i] / value_counts[temp] #ERROR HERE
  except:
    pass

  0%|          | 0/26464 [00:00<?, ?it/s]<ipython-input-12-cd4b5d788e64>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Mean Reviews"][i] = data["Mean Reviews"][i] / value_counts[temp]
<ipython-input-12-cd4b5d788e64>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Mean Score"][i] = data["Mean Score"][i] / value_counts[temp]
100%|██████████| 26464/26464 [00:07<00:00, 3670.93it/s]


In [ ]:
dates = []
for i in range(len(data)):
  date = data["Categoric Mapping"][i].split("_")[0]
  dates.append(datetime.strptime(date, "%Y-%m-%d"))
data["Date"] = dates
dates = np.unique(dates)

In [ ]:
data['Rate of Change'] = None
data['Mean Reviews'] = data['Mean Reviews'] + 1
df = data

In [ ]:
#Label = {[(1 + p) × (1 + ΔR)] x μ(S)} / σ(N)
def label_data_set(df):
    df['Rate of Change'] = None
    df['Mean Reviews'] = df['Mean Reviews'] + 1

    for i in range(len(df)):
      df["Categoric Mapping"].iloc[i] = df["Categoric Mapping"][i].split("_")[-1]

    for i in tqdm(range(len(df))):
        cat_name = df['Categoric Mapping'][i]
        lis = df.loc[df['Categoric Mapping'] == cat_name].index.tolist()

        index = lis.index(i)
        if (index == 0):
            df['Rate of Change'][i] = 0
        else:
            index_last = index - 1
            reviews = [int(df['Mean Reviews'][lis[index - 1]]), int(df['Mean Reviews'][lis[index]])]
            reviews_series = pd.Series(reviews)
            df['Rate of Change'][i] = list(reviews_series.pct_change())[1]

    df['Half Label'] = None
    
    for i in tqdm(range(len(df))):
        value = 1 + float(df['Popularity'][i])
        value2 = 1 + float(df['Rate of Change'][i])
        value = value * value2
        df['Half Label'][i] = value
        
    df['Standard Dev'] = None
    meann = {}
    for i in tqdm(list(df['Date'].unique())):
        m = df[df['Date'] == i]['Mean Reviews'].mean()
        meann[i] = m
    for i in tqdm(list(df['Date'].unique())):
        val = statistics.stdev(list(df[df['Date'] == i]['Mean Reviews']))
        df.loc[df['Date'] == i, 'Standard Dev'] = val
        
    df['Label'] = None
    for i in tqdm(range(len(df))):
        val1 = df['Half Label'][i] * df['Mean Score'][i]
        val1 = val1 / df['Standard Dev'][i]
        df['Label'][i] = val1
  
    df.drop(['Mean Reviews', 'Mean Score', 'Popularity', 'Rate of Change', 'Half Label', 'Standard Dev'], axis = 1, inplace = True)
    return df

In [ ]:
data = label_data_set(data)

<ipython-input-19-d3018e110057>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Categoric Mapping"].iloc[i] = df["Categoric Mapping"][i].split("_")[-1]
  0%|          | 0/26464 [00:00<?, ?it/s]<ipython-input-19-d3018e110057>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Rate of Change'][i] = 0
  2%|▏         | 548/26464 [00:01<00:53, 482.94it/s]<ipython-input-19-d3018e110057>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

In [ ]:
df = pd.DataFrame()

df["Category"] = "Category"
df["Sub-Category"] = "Sub-Category"
df["Sub-Sub-Category"] = "Sub-Sub-Category"
df.loc[i, "Year"] = "Year"
df.loc[i, "Month"] = "Month"
df.loc[i, "Day"] = "Day"
df.loc[i, "Date"] = "Date"
df["Label"] = "Label"

for i in range(len(data)):
  df.loc[i, "Category"] = data["Categoric Mapping"][i].split("/")[0]
  df.loc[i, "Sub-Category"] = data["Categoric Mapping"][i].split("/")[1]
  df.loc[i, "Sub-Sub-Category"] = data["Categoric Mapping"][i].split("/")[2]
  df.loc[i, "Year"] = data["Date"][i].year
  df.loc[i, "Month"] = data["Date"][i].month
  df.loc[i, "Day"] = data["Date"][i].day
  df.loc[i, "Date"] = data["Date"][i]
  df.loc[i, "Week_Day"] = data["Date"][i].strftime('%A')

  df.loc[i, "Label"] = data["Label"][i]
df.sort_index(axis = 0)

,Category,Sub-Category,Sub-Sub-Category,Year,Month,Day,Date,Label,Week_Day
0,Bags and Travel,Men Bags,Backpacks,2023,1,14,2023-01-14 00:00:00,0.053368,Saturday
1,Bags and Travel,Men Bags,Business Bags,2023,1,14,2023-01-14 00:00:00,0.046994,Saturday
2,Bags and Travel,Men Bags,Crossbody Bags,2023,1,14,2023-01-14 00:00:00,0.052711,Saturday
3,Bags and Travel,Men Bags,Messenger Bags,2023,1,14,2023-01-14 00:00:00,0.052597,Saturday
4,Bags and Travel,Men Bags,Wallets & Accessories,2023,1,14,2023-01-14 00:00:00,0.2399,Saturday
...,...,...,...,...,...,...,...,...,...
26459,Toys & Games,Sports & Outdoor Play,Inflatable Bouncers,2023,2,27,2023-02-27 00:00:00,0.813969,Monday
26460,Watches Sunglasses Jewellery,Eyewear,Men,2023,2,27,2023-02-27 00:00:00,0.042363,Monday
26461,Watches Sunglasses Jewellery,Jewellery,Women,2023,2,27,2023-02-27 00:00:00,0.005027,Monday
26462,Watches Sunglasses Jewellery,Watches,Kids,2023,2,27,2023-02-27 00:00:00,0.361823,Monday


In [ ]:
df

,Category,Sub-Category,Sub-Sub-Category,Year,Month,Day,Date,Label,Week_Day
26463,Watches Sunglasses Jewellery,Watches,Women,2023,2,27,2023-02-27 00:00:00,0.014073,Monday
0,Bags and Travel,Men Bags,Backpacks,2023,1,14,2023-01-14 00:00:00,0.053368,Saturday
1,Bags and Travel,Men Bags,Business Bags,2023,1,14,2023-01-14 00:00:00,0.046994,Saturday
2,Bags and Travel,Men Bags,Crossbody Bags,2023,1,14,2023-01-14 00:00:00,0.052711,Saturday
3,Bags and Travel,Men Bags,Messenger Bags,2023,1,14,2023-01-14 00:00:00,0.052597,Saturday
...,...,...,...,...,...,...,...,...,...
26458,Toys & Games,Remote Control & Play Vehicles,Die-Cast Vehicles,2023,2,27,2023-02-27 00:00:00,0.438588,Monday
26459,Toys & Games,Sports & Outdoor Play,Inflatable Bouncers,2023,2,27,2023-02-27 00:00:00,0.813969,Monday
26460,Watches Sunglasses Jewellery,Eyewear,Men,2023,2,27,2023-02-27 00:00:00,0.042363,Monday
26461,Watches Sunglasses Jewellery,Jewellery,Women,2023,2,27,2023-02-27 00:00:00,0.005027,Monday


In [ ]:
temp = df[(df['Category'] == "Bags and Travel") & (df['Sub-Category'] == "Men Bags") & (df['Sub-Sub-Category'] == "Business Bags")]

In [ ]:
temp

,Category,Sub-Category,Sub-Sub-Category,Year,Month,Day,Date,Label,Week_Day
1,Bags and Travel,Men Bags,Business Bags,2023,1,14,2023-01-14 00:00:00,0.046994,Saturday
596,Bags and Travel,Men Bags,Business Bags,2023,1,15,2023-01-15 00:00:00,0.039161,Sunday
1198,Bags and Travel,Men Bags,Business Bags,2023,1,16,2023-01-16 00:00:00,0.042284,Monday
1794,Bags and Travel,Men Bags,Business Bags,2023,1,17,2023-01-17 00:00:00,0.049081,Tuesday
2381,Bags and Travel,Men Bags,Business Bags,2023,1,18,2023-01-18 00:00:00,0.04818,Wednesday
2987,Bags and Travel,Men Bags,Business Bags,2023,1,19,2023-01-19 00:00:00,0.049225,Thursday
3596,Bags and Travel,Men Bags,Business Bags,2023,1,20,2023-01-20 00:00:00,0.047337,Friday
4197,Bags and Travel,Men Bags,Business Bags,2023,1,21,2023-01-21 00:00:00,0.044632,Saturday
4791,Bags and Travel,Men Bags,Business Bags,2023,1,22,2023-01-22 00:00:00,0.048821,Sunday
5392,Bags and Travel,Men Bags,Business Bags,2023,1,23,2023-01-23 00:00:00,0.053102,Monday


In [ ]:
df.to_csv("structured_dataset.csv", index = False)